<h1> 물리학에서 세 번째로 어려운 문제를 꼽자면 새로운 발견 성과를 자축할 때 쓸 적당한 와인을 찾는 것이다. </h1>
<h1> 4장의 와인 데이터를 읽어서 적당한 수의 입력 파라미터를 가지는 모델을 만들어라 </h1>

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

wine_path = "../data/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)

wineq = torch.from_numpy(wineq_numpy)

n_samples = wineq_numpy.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

wine_train = wineq[train_indices, :-1]
wine_target_train = wineq[train_indices, -1]

wine_val = wineq[val_indices, :-1]
wine_target_val = wineq[val_indices, -1]

wine_train = 0.1 * wine_train
wine_val = 0.1 * wine_val

In [73]:
seq_model = nn.Sequential(
            nn.Linear(11, 256),
            nn.Tanh(),
            nn.Linear(256, 1))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)

def loss_fn(wine_train, wine_target):
    squared_diffs = (wine_train - wine_target)**2
    return squared_diffs.mean()

def training_loop(n_epochs, optimizer, model, loss_fn, wine_train, wine_val, wine_target_train, wine_target_val):
    for epoch in range(1, n_epochs + 1):
        wine_train_out = model(wine_train)
        loss_train = loss_fn(wine_train_out, wine_target_train)

        wine_val_out = model(wine_val)
        loss_val = loss_fn(wine_val_out, wine_target_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [74]:
training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    wine_train = wine_train,
    wine_val = wine_val,
    wine_target_train = wine_target_train,
    wine_target_val = wine_target_val)


Epoch 1, Training loss 39.8454, Validation loss 39.8990
Epoch 100, Training loss 0.9632, Validation loss 0.9995
Epoch 200, Training loss 0.9441, Validation loss 0.9784
Epoch 300, Training loss 0.9371, Validation loss 0.9714
Epoch 400, Training loss 0.9305, Validation loss 0.9648
Epoch 500, Training loss 0.9242, Validation loss 0.9586
Epoch 600, Training loss 0.9182, Validation loss 0.9527
Epoch 700, Training loss 0.9125, Validation loss 0.9471
Epoch 800, Training loss 0.9072, Validation loss 0.9419
Epoch 900, Training loss 0.9021, Validation loss 0.9369
Epoch 1000, Training loss 0.8973, Validation loss 0.9322
Epoch 1100, Training loss 0.8927, Validation loss 0.9277
Epoch 1200, Training loss 0.8884, Validation loss 0.9234
Epoch 1300, Training loss 0.8843, Validation loss 0.9194
Epoch 1400, Training loss 0.8804, Validation loss 0.9156
Epoch 1500, Training loss 0.8767, Validation loss 0.9120
Epoch 1600, Training loss 0.8732, Validation loss 0.9086
Epoch 1700, Training loss 0.8698, Validat

<h2>a. 앞에서 다룬 온도 데이터와 비교할 때 훈련 시간이 얼마나 더 오래 걸리나? </h2>
<h3> - 온도 데이터의 학습 시간은 초 단위였다. 하지만 와인 데이터는 학습에 4~5분 정도 소모됬다. 와인 데이터가 학습이 5분정도 더 오래 걸린다.</h3>

<h2>b. 어떤 요소가 훈련 시간에 영향을 주는지 설명할 수 있는가? </h2>
<h3> - 데이터 셋의 크기 차이가 가장 큰 영향을 준 것 같다. 배치 단위로 분할하여 넣었다면 시간이 조금 단축 됬을 것 같다. 또한 모델의 파라미터의 숫자가 많아 학습 시간에 차이가 크게 나타났다. 와인 데이터는 약 3000개의 학습 데이터와 약 1000개의 평가 데이터를 가진다. 이들을 256개의 출력을 가진 레이어를 통과하면 가중치의 수는 대략 76만개를 넘어간다. 이는 학습해야 하는 값이 많아 지고 학습 시간을 증가시킨다.</h3>

<h2>c. 데이터셋으로 훈련시키는 동안 손실이 줄어들었는가? </h2>
<h3> - 손실 값은 Train, Validation 값 모두 줄었다.</h3>

<h2>d. 데이터셋을 어떻게 그래프로 표현할 것인가?</h2>
<h3> - 와인 데이터의 입력값인 11개의 값별로 1개의 출력값인 품질 값이 나오도록 11개의 그래프를 그릴 수 있을 것 같다.</h3>